In [34]:
import json
import nltk
import string
import pandas as pd
from langdetect import detect
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/phucpx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [49]:
# with open('data/train/evjvqa_train_lang-short-anw.json', 'r', encoding='utf-8') as f:
#     train_data = json.load(f)

# train_annotations = train_data['annotations']

with open('data/test/official_evjvqa_public_test_lang.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)

train_annotations = train_data['annotations']

In [50]:
len(train_annotations)

5015

In [51]:
lang_count = {
    "vi": 0,
    "en": 0,
    "ja": 0
}

for anno in tqdm(train_annotations):
    question = anno['question']
#     lang = detect(question)
#     anno['language'] = lang
    lang_count[anno['language']] += 1
        
lang_count

  0%|          | 0/5015 [00:00<?, ?it/s]

{'vi': 1678, 'en': 1686, 'ja': 1651}

In [52]:
def remove_punc(text):
    text = text.lower()
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)

def remove_stopword(text):
    stop_words = set(stopwords.words('english'))
  
    word_tokens = word_tokenize(text)

    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

    filtered_sentence = []

    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
            
    return ' '.join(filtered_sentence)

## Group by question type

In [53]:
en_question_type = [
    'how many', 'how is', 'what is', 'what color', 'what do', 'where are', 'where is', 'where do', 'which', 'who'
]
vi_question_type = [
    'bao nhiêu', 'như thế nào', 'làm gì', 'cái gì', 'ở đâu', 'màu gì', 'vật gì', 'phía bên nào'
]

image_ids = []
question_ids = []
languages = []
group_question_type = []
example_questions = []
example_answers = []


data_dict = {
    'how many': [], 
    'how is': [], 
    'what is': [], 
    'what color': [], 
    'what do': [], 
    'where are': [], 
    'where is': [], 
    'where do': [], 
    'which': [],
    'who': [],
    'others': [],
    'bao nhiêu': [], 
    'như thế nào': [], 
    'làm gì': [], 
    'cái gì': [], 
    'ở đâu': [], 
    'màu gì': [], 
    'vật gì': [], 
    'phía bên nào': [],
    'khác': []
}

In [54]:
for anno in tqdm(train_annotations):
    if anno['language'] == 'en':
        flag = False
        for qtype in en_question_type:
            if qtype in anno['question']:
                data_dict[qtype].append(
                    {
                        "language": "EN",
                        "question_type": qtype,
                        "question_id": anno['id'],
                        "image_id": anno['image_id'],
                        "image_path": anno['img_path'],
                        "question": anno['question'],
                        "answer": anno['answer']
                    }
                )
                flag = True
        if not flag:
            data_dict["others"].append(
                {
                    "language": "EN",
                    "question_type": "others",
                    "question_id": anno['id'],
                    "image_id": anno['image_id'],
                    "image_path": anno['img_path'],
                    "question": anno['question'],
                    "answer": anno['answer']
                }
            )
    elif anno['language'] == 'vi':
        flag = False
        for qtype in vi_question_type:
            if qtype in anno['question']:
                data_dict[qtype].append(
                    {
                        "language": "VI",
                        "question_type": qtype,
                        "question_id": anno['id'],
                        "image_id": anno['image_id'],
                        "image_path": anno['img_path'],
                        "question": anno['question'],
                        "answer": anno['answer']
                    }
                )
                flag = True
        if not flag:
            data_dict["khác"].append(
                {
                    "language": "VI",
                    "question_type": "khác",
                    "question_id": anno['id'],
                    "image_id": anno['image_id'],
                    "image_path": anno['img_path'],
                    "question": anno['question'],
                    "answer": anno['answer']
                }
            )

  0%|          | 0/5015 [00:00<?, ?it/s]

In [55]:
image_ids = []
image_paths = []
question_ids = []
languages = []
group_question_type = []
example_questions = []
example_answers = []

for qtype, samples in data_dict.items():
    for sample in samples:
        languages.append(sample['language'])
        group_question_type.append(sample['question_type'])
        question_ids.append(sample['question_id'])
        image_ids.append(sample['image_id'])
        image_paths.append(sample['image_path'])
        example_questions.append(sample['question'])
        example_answers.append(sample['answer'])

In [56]:
data_gr_dict = {
    'language': languages,
    'question_type': group_question_type,
    'question_id': question_ids,
    'image_id': image_ids,
    'image_path': image_paths,
    'question': example_questions,
    'answer': example_answers
}

df = pd.DataFrame.from_dict(data_gr_dict)

df.to_csv('./outputs/analysis/test_data.csv', index=False, encoding='utf-8')

## Analysis result

In [57]:
with open('./outputs/analysis/results-submit-16.json', 'r', encoding='utf-8') as f:
    results = json.load(f)

In [59]:
with open('data/test/official_evjvqa_public_test_lang.json', 'r', encoding='utf-8') as f:
    test = json.load(f)
    
images = test['images']
annotations = test['annotations']

In [66]:
for i, (k, v) in enumerate(results.items()):
    if annotations[i]['id'] == int(k):
        annotations[i]['answer_predicted'] = v

In [99]:
question_ids = []
image_ids = []
questions = []
image_paths = []
answer_preds = []
question_types = []

question_type = [
    'how many', 'how is', 'what is', 'what color', 'what do', 'where are', 'where is', 'where do', 'which', 'who',
    'bao nhiêu', 'như thế nào', 'làm gì', 'cái gì', 'ở đâu', 'màu gì', 'vật gì', 'phía bên nào', '\"ai ', "ai?"
]

for anno in annotations:
    question_ids.append(anno['id'])
    image_ids.append(anno['image_id'])
    questions.append(anno['question'])
    answer_preds.append(anno['answer_predicted'])
    image_paths.append(anno['img_path'])
    
    flag = False
    for qt in question_type:
        if qt in anno['question']:
            question_types.append(qt)
            flag = True
            break
    
    if not flag:
        question_types.append('others')

In [100]:
len(question_types), len(question_ids)

(5015, 5015)

In [101]:
result_df = pd.DataFrame.from_dict(
    {
        'image_id': image_ids,
        'question_type': question_types,
        'question_id': question_ids,
        'question': questions,
        'answer_pred': answer_preds,
        'image_path': image_paths
    }
)

In [102]:
dff = result_df.groupby(['image_id'])

In [103]:
dff.head()

,image_id,question_type,question_id,question,answer_pred,image_path
0,2007,how many,0,how many people are selling pho?,one,./data/test/public-test-images/00000002007.jpg
1,2007,others,1,what are the people standing in line for?,a store,./data/test/public-test-images/00000002007.jpg
2,2007,bao nhiêu,2,có bao nhiêu người đang đứng bán phở?,phở,./data/test/public-test-images/00000002007.jpg
3,2007,làm gì,3,nhóm người xếp hàng để làm gì?,mua phở,./data/test/public-test-images/00000002007.jpg
4,2007,others,4,何人がフォーを売っていますか?,フォーを売っている人が2人います,./data/test/public-test-images/00000002007.jpg
...,...,...,...,...,...,...
5000,1867,what is,5000,what is the woman in the conical hat and the g...,peeling bananas,./data/test/public-test-images/00000001867.jpg
5001,1867,others,5001,what other vegetables besides corn does the st...,tomatoes,./data/test/public-test-images/00000001867.jpg
5002,1867,how many,5002,how many people are sitting and choosing corn?,three,./data/test/public-test-images/00000001867.jpg
5003,1867,how many,5003,how many people do not walk?,two,./data/test/public-test-images/00000001867.jpg


In [104]:
result_df.to_csv('./outputs/analysis/result-analysis.csv', index=False, encoding='utf-8')

In [86]:
with open('./outputs/analysis/results-0.36bleu.json', 'r', encoding='utf-8') as f:
    result2 = json.load(f)

In [87]:
ks = []
vleus = []
for k, v in result2.items():
    ks.append(k)
    vleus.append(v)
    
rdf = pd.DataFrame.from_dict(
    {
        'id': ks,
        'answer': vleus
    }
)

rdf.to_csv('./outputs/analysis/result-2.csv', index=False, encoding='utf-8')

In [105]:
vi_question_type = [
    'bao nhiêu', 'như thế nào', 'làm gì', 'cái gì', 'ở đâu', 'màu gì', 'vật gì', 'phía bên nào'
]

for vi in vi_question_type:
    print(vi)

bao nhiêu
như thế nào
làm gì
cái gì
ở đâu
màu gì
vật gì
phía bên nào
